# AirBNB Listing

To Identify good and bad listings in the given dataset. This will help the company avoid bad listings from being put up on their platform and pull them down if already there so that customer impact and dissatisfaction is minimal

In [1]:
### Importing Required Libraries

import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection # for splitting into train and test
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn import linear_model # for logistic model
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
### Reading Dataset
dataset=pd.read_csv('listings.csv', sep=';',index_col='Unnamed: 0')
dataset.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,21,38,44,46,47,49,50,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Name,Summary,Space,Description,Experiences_Offered,Neighborhood_Overview,Notes,Transit,Access,...,Last_Review,License,Jurisdiction_Names,Cancellation_Policy,Calculated_host_listings_count,Reviews_per_Month,Geolocation,Features,Listing_Type,Rating
70285,199681.0,Private room in sunny three bedroom,This apartment in the West Rogers Park (or Wes...,Hi there traveler. Thanks for considering sta...,This apartment in the West Rogers Park (or Wes...,none,I love that my neighborhood is so quiet and lo...,NaN,Parking around here is very easy. You might wi...,"You will have your private room, and you are w...",...,2016-09-18,City registration pending,"Illinois State, Cook County, IL, CHICAGO",moderate,4.0,1.34,"41.9941700301, -87.6920295649","Host Has Profile Pic,Host Identity Verified,Re...",Good,67259.201267
44913,4216764.0,Lovely & cute apartment,Lovely apartment for a couple or a single pers...,Interior desig in accordance with the Scandina...,Lovely apartment for a couple or a single pers...,none,Quite and peaceful neighborhood in the Frederi...,"Please note that the bed is rather small (1,5 ...",Close to bus and the metro stations 'Frederiks...,"Fully equipped kitchen, 5 shelfs for your stuf...",...,2014-10-31,NaN,NaN,flexible,1.0,0.03,"55.6913945558, 12.5352884346",Host Has Profile Pic,Good,58503.191267
154025,14879686.0,Unique house with a beautiful garden,Beautiful design cosy family house with a nice...,"Beautiful garden, two level house (105 m2) nea...",Beautiful design cosy family house with a nice...,none,Nice multicultural neighbourhood with plenty o...,Since our house is designed to be child friend...,Tram and bus stops in a 5 minutes walk distanc...,House and garden,...,2016-10-02,NaN,Amsterdam,strict,1.0,0.29,"52.3689656398, 4.84983993143","Host Has Profile Pic,Is Location Exact",Good,58480.587448
281148,9879014.0,Résidence du 14 Rue des Sablons;,"50 m2, un grand lit et un canapé lit deux plac...",NaN,"50 m2, un grand lit et un canapé lit deux plac...",none,NaN,NaN,NaN,NaN,...,NaN,NaN,Paris,flexible,1.0,NaN,"48.8663711212, 2.28291441119","Host Has Profile Pic,Is Location Exact",Bad,NaN
235251,14372222.0,Darling 2 story historical spanish,"2 blocks from beach, 1 block from 2nd st",NaN,"2 blocks from beach, 1 block from 2nd st Kitch...",none,NaN,NaN,Yes,Kitchen and bath,...,NaN,NaN,NaN,flexible,1.0,NaN,"33.7595912067, -118.138810843",Host Has Profile Pic,Bad,NaN


In [3]:
### Coding Good as 1 and Bad as 0
dataset.loc[dataset.Listing_Type=='Good','Listing_Type']=1
dataset.loc[dataset.Listing_Type=='Bad','Listing_Type']=0

dataset.Listing_Type=pd.to_numeric(dataset.Listing_Type)
dataset.Listing_Type.dtype

dtype('float64')

In [4]:
### Finding dimention of data
datapoints=dataset.shape[0]
datapoints
dataset.shape

(341147, 72)

In [5]:
dataset.describe()

,Host_Response_Rate,Host_Total_Listings_Count,Latitude,Longitude,Bathrooms,Bedrooms,Beds,Price,Security_Deposit,Extra_People,Minimum_Nights,Availability_30,Availability_60,Availability_90,Availability_365,Number_of_Reviews,Calculated_host_listings_count,Reviews_per_Month,Listing_Type,Rating
count,215159.000000,339843.000000,280102.000000,280097.000000,339144.000000,339768.000000,339547.000000,275353.000000,115773.000000,280078.000000,340207.000000,340207.000000,340188.000000,340188.000000,340188.000000,340188.000000,339579.000000,256793.000000,340189.000000,251473.000000
mean,93.211130,9.618824,41.593031,-16.760623,1.249332,1.378444,1.925925,136.583102,269.040864,12.349853,3.356558,13.812624,22.789843,37.547086,164.375301,16.752525,6.932028,1.482912,0.908451,67971.466199
std,17.856188,57.517029,17.857084,58.981518,0.606616,0.941545,1.495883,153.269083,170.807709,33.286100,25.004775,527.657227,22.308217,33.819020,140.356031,32.306328,42.281312,1.747969,0.288389,6161.660098
min,0.000000,0.000000,-38.192248,-122.833981,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.010000,0.000000,14631.791267
25%,98.000000,1.000000,40.410061,-73.909358,1.000000,1.000000,1.000000,53.000000,150.000000,0.000000,1.000000,0.000000,0.000000,0.000000,13.000000,1.000000,1.000000,0.320000,1.000000,65811.731267
50%,100.000000,1.000000,45.518834,2.110237,1.000000,1.000000,1.000000,86.000000,200.000000,0.000000,2.000000,5.000000,16.000000,32.000000,144.000000,4.000000,1.000000,0.880000,1.000000,69475.792788
75%,100.000000,3.000000,51.510523,4.862231,1.000000,2.000000,2.000000,150.000000,350.000000,15.000000,3.000000,19.000000,43.000000,70.000000,312.000000,18.000000,2.000000,2.030000,1.000000,73087.151267
max,100.000000,1114.000000,55.994889,153.636919,10.000000,30.000000,60.000000,999.000000,999.000000,999.000000,10000.000000,73128.453317,60.000000,90.000000,365.000000,574.000000,752.000000,223.000000,1.000000,73151.002116


# Cleaning Data
## Droping columns

In [6]:
# ID column is not required
dataset.drop(columns=['ID'],inplace=True)

### Location Related Column

We will drop location related column. Since there is no sence of doing one hot encoding for so many states.

In [7]:
# Location Related Columns 
locationList=['Host_Location','Host_Neighbourhood','Street',
              'Neighbourhood','State','Zipcode',
              'City','Market','Smart_Location','Country_Code','Country',
              'Geolocation' ,'Latitude','Longitude'
             ]#
dataset=dataset.drop(columns=locationList)

### Text Based Column
We will do feature engg using NLP for these text based column

As of now lets delete these columns

In [8]:
# NLP
nlpColumns=['Name','Summary','Space','Description',
            'Neighborhood_Overview','Transit','Access',
            'Interaction','House_Rules','Host_About','Host_Name','Neighbourhood_Cleansed']


dataset=dataset.drop(columns=nlpColumns)



### Missing Values greater than 30%

We will drop the columns which has missing values greater than 30 %

In [9]:
# missing value in percentage
missingDataInfo=pd.DataFrame([dataset.isnull().sum()*100/datapoints,dataset.dtypes]).T
missingDataInfo.columns=['MissingValues','DataType']


# Check Missing Value columns in percentage
missingDataAboveThresholdColumnList=missingDataInfo[missingDataInfo['MissingValues']>30].index

dataset=dataset.drop(columns=missingDataAboveThresholdColumnList)
missingDataAboveThresholdColumnList # Lets delete these columns as of now

Index(['Notes', 'Host_Response_Rate', 'Host_Acceptance_Rate',
       'Neighbourhood_Group_Cleansed', 'Square_Feet', 'Weekly_Price',
       'Monthly_Price', 'Security_Deposit', 'Cleaning_Fee', 'Has_Availability',
       'License', 'Jurisdiction_Names'],
      dtype='object')

## Feature Engginearing

### Date Column

Difference between today's date and hosted date.

In [10]:
dateList=['Host_Since','Calendar_last_Scraped','First_Review','Last_Review']
dataset[dateList].head()

,Host_Since,Calendar_last_Scraped,First_Review,Last_Review
70285,2011-08-15,2017-05-10,2011-08-27,2016-09-18
44913,2012-02-10,2017-06-15,2014-10-31,2014-10-31
154025,2012-10-23,2017-04-02,2016-09-07,2016-10-02
281148,2015-12-08,2017-04-05,NaN,NaN
235251,2015-09-13,2017-05-03,NaN,NaN


In [11]:
for dateColumn in dateList:
    print(dateColumn)
    dataset[dateColumn] = pd.to_datetime(dataset[dateColumn],format='%Y-%m-%d',errors='coerce')

Host_Since
Calendar_last_Scraped
First_Review
Last_Review


In [12]:
dataset['Host_Since'].head()

70285    2011-08-15
44913    2012-02-10
154025   2012-10-23
281148   2015-12-08
235251   2015-09-13
Name: Host_Since, dtype: datetime64[ns]

In [13]:
for dateColumn in dateList:
    dataset[dateColumn+'_DAYSSINCETODAY']=(pd.datetime.now()-dataset[dateColumn]).dt.days
    

In [14]:
dataset.drop(columns=dateList,inplace=True)

In [15]:
dataset.columns

Index(['Experiences_Offered', 'Host_Response_Time', 'Host_Listings_Count',
       'Host_Total_Listings_Count', 'Property_Type', 'Room_Type',
       'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed_Type',
       'Amenities', 'Price', 'Guests_Included', 'Extra_People',
       'Minimum_Nights', 'Maximum_Nights', 'Calendar_Updated',
       'Availability_30', 'Availability_60', 'Availability_90',
       'Availability_365', 'Number_of_Reviews', 'Cancellation_Policy',
       'Calculated_host_listings_count', 'Reviews_per_Month', 'Features',
       'Listing_Type', 'Rating', 'Host_Since_DAYSSINCETODAY',
       'Calendar_last_Scraped_DAYSSINCETODAY', 'First_Review_DAYSSINCETODAY',
       'Last_Review_DAYSSINCETODAY'],
      dtype='object')

### Calender Updated 

There are so many entries like 6 days ago, 7 days ago, 5 months ago.

Lets restrict these entries only by day, month, week.

In [16]:
dataset['Calendar_Updated'].head()

70285        6 days ago
44913      5 months ago
154025       3 days ago
281148    16 months ago
235251     9 months ago
Name: Calendar_Updated, dtype: object

In [17]:
dataset['Calendar_Updated']=dataset['Calendar_Updated'].str.extract(r'(day|month|week|never)')

In [18]:
dataset['Calendar_Updated'].head()

70285       day
44913     month
154025      day
281148    month
235251    month
Name: Calendar_Updated, dtype: object

In [19]:
dataset['Calendar_Updated'].value_counts()

day      132058
week     105671
month     99652
never      2807
Name: Calendar_Updated, dtype: int64

### Cancellation Policy

Lets the cancellation policy only be strict,flexible, moderate.

Super_strict_60,super_strict_30 will convert into strict.

In [20]:
dataset['Cancellation_Policy'].value_counts()

strict                 128187
flexible               109382
moderate                85297
moderate_new             5955
strict_new               5566
flexible_new             4193
super_strict_60           950
super_strict_30           605
super_strict_30_new        27
super_strict_60_new        21
no_refunds                  3
long_term                   2
Name: Cancellation_Policy, dtype: int64

In [21]:
dataset['Cancellation_Policy']=dataset['Cancellation_Policy'].str.extract(r'(flexible|moderate|strict|no)')

In [22]:
dataset['Cancellation_Policy'].value_counts()

strict      135356
flexible    113575
moderate     91252
no               3
Name: Cancellation_Policy, dtype: int64

## Missing Value Imputation

Lets impute missing values by median

In [23]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numColumns=dataset.select_dtypes(include=numerics).columns
numColumns

Index(['Host_Total_Listings_Count', 'Bathrooms', 'Bedrooms', 'Beds', 'Price',
       'Extra_People', 'Minimum_Nights', 'Availability_30', 'Availability_60',
       'Availability_90', 'Availability_365', 'Number_of_Reviews',
       'Calculated_host_listings_count', 'Reviews_per_Month', 'Listing_Type',
       'Rating', 'Host_Since_DAYSSINCETODAY',
       'Calendar_last_Scraped_DAYSSINCETODAY', 'First_Review_DAYSSINCETODAY',
       'Last_Review_DAYSSINCETODAY'],
      dtype='object')

In [24]:
# % of missing values
dataset[numColumns].isnull().sum()*100/datapoints 

Host_Total_Listings_Count                0.382240
Bathrooms                                0.587137
Bedrooms                                 0.404225
Beds                                     0.469006
Price                                   19.286114
Extra_People                            17.901081
Minimum_Nights                           0.275541
Availability_30                          0.275541
Availability_60                          0.281110
Availability_90                          0.281110
Availability_365                         0.281110
Number_of_Reviews                        0.281110
Calculated_host_listings_count           0.459626
Reviews_per_Month                       24.726584
Listing_Type                             0.280817
Rating                                  26.286029
Host_Since_DAYSSINCETODAY                0.112268
Calendar_last_Scraped_DAYSSINCETODAY     0.281110
First_Review_DAYSSINCETODAY             24.726584
Last_Review_DAYSSINCETODAY              24.705772


In [25]:
#dataset['Security_Deposit']=dataset['Security_Deposit'].fillna(0)
dataset['Extra_People']=dataset['Extra_People'].fillna(0)

# As of now lets impute numeric columns by MEDIAN
dataset[numColumns]=dataset[numColumns].fillna(dataset.median())

## One Hot Encoding

In [26]:
categoricalColForOHE=['Host_Response_Time','Room_Type','Bed_Type'] #,'Listing_Type'
df_OneHot=pd.get_dummies(dataset[categoricalColForOHE], 
                                 prefix=None, prefix_sep='_', 
                                 dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)
df_OneHot.head()

,Host_Response_Time_Australia,Host_Response_Time_Austria,Host_Response_Time_Belgium,Host_Response_Time_Canada,Host_Response_Time_Denmark,Host_Response_Time_Ireland,Host_Response_Time_Spain,Host_Response_Time_United Kingdom,Host_Response_Time_United States,Host_Response_Time_a few days or more,...,Bed_Type_69.0,Bed_Type_8.0,Bed_Type_84.0,Bed_Type_89.0,Bed_Type_90.0,Bed_Type_Airbed,Bed_Type_Couch,Bed_Type_Futon,Bed_Type_Pull-out Sofa,Bed_Type_Real Bed
70285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
44913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
154025,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
281148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
235251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
dataset.drop(columns=categoricalColForOHE,inplace=True)
dataset=pd.concat([dataset,df_OneHot],axis=1)

## Comma separated Values

We will process CSV columns. i.e. 
* Amenities, 
* Features


Lets replace space by _ and split values where , occurs


Example Amenities are TV,Internet,Wireless Internet, AC.

Then there will be 4 columns TV,Internet,Wireless Internet, AC. And Each row will have boolean value indicating whether TV present or not.. so on

In [28]:
dataset[['Amenities','Features']].head()

,Amenities,Features
70285,"TV,Internet,Wireless Internet,Air conditioning...","Host Has Profile Pic,Host Identity Verified,Re..."
44913,"Internet,Kitchen,Pets live on this property,Do...",Host Has Profile Pic
154025,"TV,Internet,Wireless Internet,Kitchen,Heating,...","Host Has Profile Pic,Is Location Exact"
281148,"TV,Cable TV,Internet,Wireless Internet,Air con...","Host Has Profile Pic,Is Location Exact"
235251,NaN,Host Has Profile Pic


In [29]:
dataset['Amenities']=dataset['Amenities'].str.replace(' ','_')
dataset['Amenities']=dataset['Amenities'].str.replace('\"','')
dataset['Amenities']=dataset['Amenities'].str.replace(',',' ')
dataset['Amenities']=dataset['Amenities'].fillna('')
dataset['Amenities']=dataset['Amenities'].str.replace('-','_')


dataset['Features']=dataset['Features'].str.replace(' ','_')
dataset['Features']=dataset['Features'].str.replace('\"','')
dataset['Features']=dataset['Features'].str.replace(',',' ')
dataset['Features']=dataset['Features'].fillna('')
dataset['Features']=dataset['Features'].str.replace('-','_')

In [30]:
dataset[['Amenities','Features']].head()

,Amenities,Features
70285,TV Internet Wireless_Internet Air_conditioning...,Host_Has_Profile_Pic Host_Identity_Verified Re...
44913,Internet Kitchen Pets_live_on_this_property Do...,Host_Has_Profile_Pic
154025,TV Internet Wireless_Internet Kitchen Heating ...,Host_Has_Profile_Pic Is_Location_Exact
281148,TV Cable_TV Internet Wireless_Internet Air_con...,Host_Has_Profile_Pic Is_Location_Exact
235251,,Host_Has_Profile_Pic


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectFeatures = CountVectorizer()
vectFeatures.fit(dataset['Features'])

featuresSplit=vectFeatures.transform(dataset['Features'])
#featuresSplit.toarray()

vectAmenities = CountVectorizer()
vectAmenities.fit(dataset['Amenities'])

amenitiesSplit=vectAmenities.transform(dataset['Amenities'])
#amenitiesSplit.toarray()


In [32]:
featuresCSV=pd.DataFrame(featuresSplit.toarray(),columns=vectFeatures.get_feature_names())

amenitiesCSV=pd.DataFrame(amenitiesSplit.toarray(),columns=vectAmenities.get_feature_names())

csvDF=pd.concat([featuresCSV,amenitiesCSV],axis=1)



In [33]:
dataset.drop(columns=['Amenities','Features'],inplace=True)

In [34]:
dataset=pd.concat([dataset.reset_index(drop=True),csvDF],axis=1)


In [35]:
dataset.head()

,Experiences_Offered,Host_Listings_Count,Host_Total_Listings_Count,Property_Type,Accommodates,Bathrooms,Bedrooms,Beds,Price,Guests_Included,...,washer_,waterfront,wheelchair_accessible,wide_clearance_to_bed,wide_clearance_to_shower_and_toilet,wide_doorway,wide_hallway_clearance,window_guards,wireless_intercom,wireless_internet
0,none,4.0,4.0,Apartment,2.0,1.0,1.0,1.0,50.0,1.0,...,0,0,0,0,0,0,0,0,0,1
1,none,1.0,1.0,Apartment,2.0,1.0,1.0,1.0,371.0,1.0,...,0,0,0,0,0,0,0,0,1,0
2,none,1.0,1.0,Apartment,4.0,1.0,2.0,4.0,145.0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,none,1.0,1.0,Apartment,4.0,1.0,1.0,2.0,115.0,1.0,...,0,0,1,0,0,0,0,0,1,1
4,none,1.0,1.0,House,1.0,1.0,1.0,1.0,200.0,1.0,...,0,0,0,0,0,0,0,0,0,0


# Model

## Splitting Data

In [36]:
dropColList=['Listing_Type']
pStar_List=[] # Unimportant columns. As of now it is empty

In [37]:
dropColList.extend(pStar_List)

In [38]:
# Split-out validation dataset
X = dataset.select_dtypes(include=numerics).drop(columns=dropColList).values
Y = dataset['Listing_Type'].values
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X_Columns=dataset.select_dtypes(include=numerics).drop(columns=dropColList).columns
validation_size = 0.30
seed = 100
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

##### Higher Order Poly

In [39]:
X_train*X_train

array([[5.03627731e-04, 6.40000000e-01, 1.11111111e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [3.22321748e-06, 1.00000000e-02, 1.00000000e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [7.25223933e-06, 1.00000000e-02, 1.11111111e-03, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [3.22321748e-06, 1.00000000e-02, 1.11111111e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.05804370e-07, 1.00000000e-02, 1.11111111e-03, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [3.22321748e-06, 1.00000000e-02, 1.77777778e-02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

## Logistic Regression

In [40]:
# Algorithms
model_LR=sklearn.linear_model.LogisticRegression(class_weight='balanced')


# fitting model
model_LR.fit(X_train,Y_train)

#### Making Prediction ####

trainResult_LR=model_LR.predict(X_train)
testResult_LR=model_LR.predict(X_test)



### Evaluation

In [41]:


# Train

print(sklearn.metrics.accuracy_score(Y_train, trainResult_LR))
print(sklearn.metrics.confusion_matrix(Y_train, trainResult_LR))
print(sklearn.metrics.classification_report(Y_train, trainResult_LR))
print('--------------------------------------------------------------')

# Test
print(sklearn.metrics.accuracy_score(Y_test, testResult_LR))
print(sklearn.metrics.confusion_matrix(Y_test, testResult_LR))
print(sklearn.metrics.classification_report(Y_test, testResult_LR))


0.7732514803058601
[[ 20017   1752]
 [ 52396 164637]]
             precision    recall  f1-score   support

        0.0       0.28      0.92      0.43     21769
        1.0       0.99      0.76      0.86    217033

avg / total       0.92      0.77      0.82    238802

--------------------------------------------------------------
0.7708437148859251
[[ 8541   834]
 [22619 70351]]
             precision    recall  f1-score   support

        0.0       0.27      0.91      0.42      9375
        1.0       0.99      0.76      0.86     92970

avg / total       0.92      0.77      0.82    102345



### Finding Important columns

In [42]:
tempList=['Intercept']
tempList.extend(X_Columns)

params = np.append(model_LR.intercept_,model_LR.coef_)
predictions = model_LR.predict(X_train)

newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))

MSE = mean_squared_error(Y_test, testResult_LR)

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

LRSummaryDF = pd.DataFrame()
LRSummaryDF["Cols"],LRSummaryDF["Coefficients"],LRSummaryDF["Standard Errors"],LRSummaryDF["t values"],LRSummaryDF["Probabilites"] = [tempList,params,sd_b,ts_b,p_values]


LRSummaryDF['pStar']='.'
LRSummaryDF.loc[LRSummaryDF['Probabilites'] < 0.01,'pStar']='*'
LRSummaryDF.loc[LRSummaryDF['Probabilites'] < 0.01,'pStar']='* *'
LRSummaryDF.loc[LRSummaryDF['Probabilites'] < 0.001,'pStar']='* * *'

display(LRSummaryDF)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':


,Cols,Coefficients,Standard Errors,t values,Probabilites,pStar
0,Intercept,-1.8963,7.000000e-03,-289.165,0.000,* * *
1,Host_Total_Listings_Count,4.5454,3.100000e-02,148.066,0.000,* * *
2,Bathrooms,-0.4866,1.500000e-02,-31.554,0.000,* * *
3,Bedrooms,-1.3173,7.900000e-02,-16.694,0.000,* * *
4,Beds,2.3105,7.600000e-02,30.311,0.000,* * *
5,Price,-0.1827,7.000000e-03,-24.965,0.000,* * *
6,Extra_People,1.7665,3.100000e-02,56.635,0.000,* * *
7,Minimum_Nights,0.1768,2.460000e-01,0.719,0.472,.
8,Availability_30,1.6862,2.330000e-01,7.242,0.000,* * *
9,Availability_60,-1.5090,9.000000e-03,-164.297,0.000,* * *


In [43]:
pStar_List=LRSummaryDF[LRSummaryDF['pStar']=='.'].Cols

# Go to again 3.1 Splitting Data. 

In [45]:
# Important Features
LRSummaryDF[LRSummaryDF['pStar']=='* * *'].Cols

0                                 Intercept
1                 Host_Total_Listings_Count
2                                 Bathrooms
3                                  Bedrooms
4                                      Beds
5                                     Price
6                              Extra_People
8                           Availability_30
9                           Availability_60
10                          Availability_90
11                         Availability_365
12                        Number_of_Reviews
13           Calculated_host_listings_count
14                        Reviews_per_Month
15                                   Rating
16                Host_Since_DAYSSINCETODAY
17     Calendar_last_Scraped_DAYSSINCETODAY
18              First_Review_DAYSSINCETODAY
19               Last_Review_DAYSSINCETODAY
20                     host_has_profile_pic
21                   host_identity_verified
22                        host_is_superhost
23                         insta